In [ ]:
!pip install telethon pandas nest_asyncio -q

import os
import re
import asyncio
import nest_asyncio
import pandas as pd
from telethon import TelegramClient

nest_asyncio.apply()


api_id = ***
api_hash = ***
phone = ***


CHANNEL_USERNAME = "kunuzofficial"
OUTPUT_FILE = "Uzbek_Cyrillic_400.txt"


def clean_text(text):
    
    text = re.sub(r"http\S+|www\.\S+|t\.me/\S+|\S+\.(uz|ru|com|kr)\S*", "", text, flags=re.IGNORECASE)
    
    
    text = re.sub(r"[@#]\S+", "", text)
    text = re.sub(r"[*•▪️]+", "", text)
    text = re.sub(r"[\"”“„»«]", "", text)
    text = re.sub(r"[\(\)\[\]<>]", "", text)
    
    
    text = re.sub(r"(?i)obuna|подпис|subscribe|reklama|kanal|канал|telegram", "", text)
    
    
    text = re.sub(r"\s+", " ", text).strip()
    return text


async def fetch_messages():
    client = TelegramClient("uzbek_cyrillic_session", api_id, api_hash)
    await client.start(phone)
    print(f"📥 Fetching messages from {CHANNEL_USERNAME}...")

    clean_sentences = []
    limit = 10000  

    async for message in client.iter_messages(CHANNEL_USERNAME, limit=limit):
        if message.text:
            text = clean_text(message.text)

            
            if len(text.split()) > 3 and not any(char.isdigit() for char in text):
                clean_sentences.append(text)

        
        if len(clean_sentences) >= 400:
            break

    await client.disconnect()
    print(f"✅ Collected {len(clean_sentences)} clean sentences.")
    return clean_sentences


async def main():
    sentences = await fetch_messages()
    df = pd.DataFrame(sentences, columns=["sentence"])
    df.to_csv(OUTPUT_FILE, index=False, header=False, encoding="utf-8")
    print(f"💾 Saved {len(sentences)} sentences to {OUTPUT_FILE}")

await main()


#Please enter the code you received: ***
#Signed in successfully as ***; remember to not break the ToS or you will risk an account ban!
#📥 Fetching messages from kunuzofficial...
#✅ Collected 400 clean sentences.
#💾 Saved 400 sentences to Uzbek_Cyrillic_400.txt